### Exercício de modelos mistos -- Problem 2

#### Beltrán Liniers  nº57646

#### Luís Santos nº57470

No presente trabalho será abordado modelos mistos. Estes consistem numa combinação de diferentes modelos de classificação, denominados de experts, para realizarem o processo de previsão [1].
O dataset é composto por 30 variáveis independentes e 569 observações. Sendo a target variable binária que indica se possuí cancro da mama ou não. 
Inicia-se com a definição de uma série de funções que vão ajudar a automatizar os processos.


In [37]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from scipy.special import softmax

### Functions definition

In [38]:
def load_data():
    data = load_breast_cancer()
    X = data.data
    y = data.target
    features = data.feature_names
    target = data.target_names
    return X,y,features,target

In [39]:
def preprocess_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=12)
    return X_train, X_test, y_train, y_test

No processo todo de classificação serão utilizados três conjuntos de dados vindos do mesmo dataset inicial: train, validation e test. Este conjunto train será utilizado para treinar o modelo. Posteriormente, os dados de validação serão usados para calcular os accuracy de cada expert, os quais vão ser usados para criar as ponderações com a função softmax. Finalmente, o conjunto de test será utilizado para predizer e calcular o accuracy final.

Previamente é necessário realizar uma standardização dos dados, devido à grande diferença entre as medidas das features. Esta normalização será feita no conjunto de treino. Posteriormente, no momento de uso do validation set e test set, estes também serão normalizados.


In [40]:
X, y, features, target = load_data()
X_train,X_test,y_train,y_test = preprocess_data(X,y)

A função `train test models` permite testar uma serie de modelos, onde os Accuracy scores de cada modelo serão utilizados para calcular uma média ponderada. Esta é multiplicada pelos outputs dos modelos. A este resultado obtido é aplicado a função `round` obtendo assim uma matriz com valores binários de 0 e 1. 

Ao se utilizar os accuracy para calcular esta média, está-se a concluir que se terão modelos con maior capacidade de classificação e importância que outros. Pelo que, num sistema de votação estes terão um maior peso de voto, no label a atribuir ao exemplo [2]. 


In [41]:
def train_test_models(experts,X,y):
    scores = []
    y_preds = None
    X_train,X_validation,y_train,y_validation = train_test_split(X,y,test_size=0.33,random_state=12)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    
    for expert in experts:
        expert.fit(X_train,y_train)
        X_validation = scaler.transform(X_validation)
        
        score = expert.score(X_validation,y_validation)
        scores.append(score)
    return scores
    

In [42]:
def weighted_avg_ensemble(experts,scores,X_test):
    y_preds = None
    scaler = StandardScaler()
    scaler.fit(X_test)
    for expert in experts:
        X_test = scaler.transform(X_test)
        y_pred = [expert.predict(X_test)]
        if y_preds is None:
            y_preds = y_pred
        else:
            y_preds = np.concatenate((y_preds,y_pred))
    
    output = np.round(np.dot(softmax(scores),y_preds))
    return output

Por outro lado, a função `expert_optimizer` encontra a melhor combinação de modelos expert a ser testados, assim como o número ótimo deles que maximizam o accuracy. 

In [43]:
def expert_optimizer(experts_list,X_train,X_test,y_train,y_test,num_experts = 2):
    
    scores_arr = []
    j=0
    while j <= (len(experts_list)-num_experts):
        scores = train_test_models(experts_list[j:j+num_experts], X_train, y_train)
        output_preds = weighted_avg_ensemble(experts_list[j:j+num_experts],scores,X_test)
        new_acc = accuracy_score(y_test,output_preds)
        scores_arr.append({'acc':new_acc,'experts':experts_list[j:j+num_experts]})
        j+=num_experts
    df_scores = pd.DataFrame(scores_arr)
    #print(df_scores)
    return df_scores.loc[df_scores['acc']==df_scores['acc'].max()]
            

##### Experts Decision Trees

O primeiro teste vai ser uma combinação de duas árvores de decisão, com a diferença no híperparâmetro *criterion*: gini e entrópia. Como output da função `train_test_models` obteve-se a matriz com valores binários que corresponde aos labels obtidos com a *Weighted Average Ensemble*. 

A Accuracy obtida co o conjunto de teste é superior a 0.88.

In [55]:
models = [DecisionTreeClassifier(criterion='gini'),DecisionTreeClassifier(criterion='entropy')]
scores = train_test_models(models,X_train,y_train)
output_DT = weighted_avg_ensemble(models,scores,X_test)
output_DT

array([0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0.])

In [56]:
print("O accuracy com dois experts de decision tree é: ", accuracy_score(y_test,output_DT))

O accuracy com dois experts de decision tree é:  0.8882978723404256


##### Experts SVM

Experimentou-se dois SVM’s como experts. Sendo que no primeiro expert foi definido um poly kernel de grau 2. E no segundo expert foi especificado um rbf kernel. O resultado obtido foi inferior ao accuracy conseguido com as Árvores de Decisão, sendo de 0.835.

In [47]:
models = [SVC(kernel='poly', C=1, gamma=0.1, degree=2),SVC(kernel='rbf', C=1, gamma=0.1)]
scores = train_test_models(models,X_train,y_train)
output_SVM = weighted_avg_ensemble(models,scores,X_test)
output_SVM

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0.])

In [48]:
accuracy_score(y_test,output_SVM)
print("Accuracy score with 2 experts both SVM: ", accuracy_score(y_test,output_SVM))

Accuracy score with 2 experts both SVM:  0.8351063829787234


##### Experts Logistic Regression

Posteriormente, utilizou-se dois experts de Regressão logística, estes diferenciam-se pelo solver utilizado: liblinear e lbfgs.

Foi obtida a accuracy mais elevada dos três ensemble models.

In [49]:
models = [LogisticRegression(solver='liblinear', C=1, max_iter=5000), LogisticRegression(solver='lbfgs', C=1,max_iter=5000)]
scores = train_test_models(models,X_train,y_train)
output_LR = weighted_avg_ensemble(models,scores,X_test)
output_LR

array([0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0.])

In [50]:
accuracy_score(y_test,output_LR)
print("Accuracy score with 2 experts both Logistic Regression: ", accuracy_score(y_test,output_LR))

Accuracy score with 2 experts both Logistic Regression:  0.9574468085106383


##### Combinação de Experts com diferente número de níveis e classes de modelos.

Seguidamente, para efeitos de análise e comparação entre modelos, desenvolveu-se a função `expert_optimizer`, que encontra a melhor combinação de experts. 

No conjunto de dados utilizado, os melhores resultados foram obtidos, de forma clara, com ambos modelos da regressão logística. Como tal, decidiu-se não incorporá-los no conjunto de experts a testar.

In [51]:
models = [DecisionTreeClassifier(criterion='gini'), 
          DecisionTreeClassifier(criterion='entropy'), 
          SVC(kernel='poly', C=1, gamma=0.1, degree=2),
          SVC(kernel='rbf', C=1, gamma=0.1)]

optimal_model = expert_optimizer(models, X_train, X_test, y_train, y_test,3)
print(optimal_model)

        acc                                            experts
0  0.930851  [DecisionTreeClassifier(ccp_alpha=0.0, class_w...


Assim, deste processo o melhor resultado obtido foi com os experts SVM com um rbf kernel e uma Árvore de Decisão com o critério de entropia. Tendo sido o accuracy score de 0.93.

In [52]:
optimal_model['experts'].values[0]

[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'),
 SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=2, gamma=0.1

In [53]:
print("A combinação ótima de experts é formada pelos modelos: {}".format([optimal_model['experts'].values[0][i] for i in range(len(optimal_model['experts'].values[0]))]))
print("\nO accuracy do modelo mixto ótimo é: ", optimal_model['acc'].values[0])

A combinação ótima de experts é formada pelos modelos: [DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision

A principal vantagem dos modelos mistos é que possibilita o melhoramento da performance, em comparação a um modelo único. Como foi visto, com duas Árvores de Decisão obteve um accuracy de 0.88, e o modelo com dois SVM, 0.83. Mas com a combinação de ambos alcançou-se um accuracy score de 0.93.

Por outro lado, não é uma técnica que garanta sempre melhores resultados, como foi observado com a combinação de duas Logistic Regression obteve-se o melhor accuracy score de todos os modelos.


#### References

[1] Ensemble Models. (2022). Retrieved 19 March 2022, from https://towardsdatascience.com/ensemble-models-5a62d4f4cb0c

[2] Brownlee, J. (2022). How to Develop a Weighted Average Ensemble With Python. Retrieved 20 March 2022, from https://machinelearningmastery.com/weighted-average-ensemble-with-python/